In [1]:
from itertools import combinations
from tqdm.auto import tqdm
import numpy as np
import math
import sample as sampler
import utils
import attacker
from model import BackBone

## Dataset

In [2]:
# model path, where your model is 
model_path = "/mnt/e/BaiduNet/ms1mv3_arcface_r100_fp16"
# fatial data path(upper diretory)
data_fei_retinaface_path = "/mnt/e/Downloads/FEI/originalimages_retinaface_122"
data_color_feret_path = "/mnt/e/Downloads/colorferet/images_retina"
data_fatial_path = data_color_feret_path
# model parameters, need to revise if using different model
# for example, ms1mv3_arcface_r100_fp16 -> "r100", "fp16": True
kwargs = {"name": "r100", "dropout":0.0, "fp16": True, "num_features": 512}

In [3]:
backbone_model = BackBone(platform="pytorch", backbone_path=f"{model_path}/backbone.pth", **kwargs)

In [4]:
import tempfile
# tempdir: where to temporarily store fatial embeddings
tempdir = '/tmp/' + model_path.split('/')[-1].split('.')[0]
tempdir

'/tmp/ms1mv3_arcface_r100_fp16'

In [5]:
from torchvision import transforms
from dataset.fatial_dataset import FatialDataset

fatial_dataset = FatialDataset(data_fatial_path, transform=transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    lambda x: x * 2 - 1]), target_transform=lambda label: str(label))

In [6]:
from dataset.fatial_dataset_embedding_dict import FatialDataEmbeddingsDict
dataset_dict = FatialDataEmbeddingsDict(file_folder=tempdir, data_set=fatial_dataset, file_raw_name="labels.pickle")
dataset_dict.dump_embeddings(backbone_model, 10)

In [7]:
identity_list = list(dataset_dict.keys())
print(len(identity_list))

1123


## Parameters

In [31]:
# codeword parameters
dimension = 512
alpha = 16
n = 511
error_rate = 0.36    # angle(w + noise_1, w + noise_2) 
                    # approximate 2*arcsin(error_rate * \sqrt(2) / \sqrt(1 + error_rate^2) / 2)
print("Estimated angle between two noisy templates: {}".format(2 * np.arcsin(error_rate * np.sqrt(2) / np.sqrt(1 + error_rate**2) / 2) * 180 / np.pi))

Estimated angle between two noisy templates: 27.715342269093377


## Secure sketch true accept rate

In [13]:
identity_list = list(dataset_dict.keys())
print(len(identity_list))

import ironmask
import math
success_num = 0
whole_num = 0
index_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # according to labels.csv, if the sequences is p02, p03, ..., then index of 0 is p02, index of 1 is p03 and etc
index_list = [0, 2, 3]
for each_identity in tqdm(identity_list):
    for index in index_list:
        tmpcs = ironmask.sample_codeword(dimension, alpha)
        tmpw = dataset_dict[each_identity][index]
        # tmpw = sampler._generate_random_unit_vector_nearby(tmpw, math.tan(56.3228 * math.pi / 180))
        tmp_sketch = ironmask.generate_secure_sketch(tmpw, tmpcs)
        for indexj in index_list:
            if index==indexj: continue
            tmpw = dataset_dict[each_identity][indexj]
            candidate = tmp_sketch @ tmpw
            candidate = ironmask.decode_codeword(candidate, dimension, alpha)
            if np.allclose(candidate, tmpcs):
                success_num += 1
            whole_num += 1

1123


  0%|          | 0/1123 [00:00<?, ?it/s]

In [16]:
print("success_num: ", success_num, "\twhole_num: ", whole_num, "success rate: ", "{:.4f}%".format(success_num / whole_num * 100))

success_num:  6045 	whole_num:  6738 success rate:  89.7150%


## Mean angle of two templates

In [12]:
# generate puzzle
import ironmask
import itertools
# cs = []
# isometric_matrixes = []
index_list = [0, 2, 3]
angles_sum = 0
for each_identity in tqdm(identity_list):
    # each_isometric_matrixes = []
    # each_cs = []
    for indexi, indexj in itertools.combinations(index_list, 2):
        angles_sum += utils.get_angle_of_two_vectors(dataset_dict[each_identity][indexi], dataset_dict[each_identity][indexj])
    # for index in index_list:  # according to labels.csv, if the sequences is p02, p03, ..., then index of 0 is p02, index of 1 is p03 and etc
    #     tmpcs = ironmask.sample_codeword(dimension, alpha)
    #     tmpw = dataset_dict[each_identity][index]
    #     # tmpw = sampler._generate_random_unit_vector_nearby(tmpw, math.tan(56.3228 * math.pi / 180))
    #     each_isometric_matrixes.append(ironmask.generate_secure_sketch(dataset_dict[each_identity][index], tmpcs))
    #     each_cs.append(tmpcs)
    # cs.append(cs)
    # isometric_matrixes.append(each_isometric_matrixes)
angle_mean = angles_sum / (len(index_list) * (len(index_list) - 1) // 2 * len(identity_list))
print(angle_mean)

  0%|          | 0/1123 [00:00<?, ?it/s]

39.68718703137075


## solving original template if sampled matrix is correct

ensure the matrix generated by linear equation sampler is "correct" as in Definition 4.1 in paper.

lsa solver

In [ ]:
runtimes_vec = []
success_times = 0
whole_iteration_times = 80000
for each_identity in (pbar:=tqdm(identity_list)):
    each_isometric_matrixes = []
    each_cs = []
    for index in [0, 2, 3]:
        tmpcs = ironmask.sample_codeword(dimension, alpha)
        each_isometric_matrixes.append(ironmask.generate_secure_sketch(dataset_dict[each_identity][index], tmpcs))
        each_cs.append(tmpcs)
    assume_vector, b, run_times = attacker.solve_puzzle_with_n_matrix_known_places(each_isometric_matrixes, each_cs, dimension, alpha,  threshold=49, max_rtimes=whole_iteration_times//len(identity_list), algorithm="LSA", disable_tqdm=False, k_each_matrix=200, error_rate = error_rate * 3.0, return_runtimes = True)
    if b is not None and (np.allclose(b, each_cs[0]) or np.allclose(b, -each_cs[0])):
        success_times += 1
    pbar.set_postfix({"Success Rate": success_times / (pbar.n + 1)})
    runtimes_vec.append(run_times)

In [20]:
import pickle
with open("runtimes_vec_feret.pickle", "wb") as f:
    pickle.dump(runtimes_vec, f)
with open("success_times_feret.pickle", "wb") as f:
    pickle.dump(success_times, f)

In [21]:
import pickle
from utils import subset_n_a_alpha

with open("runtimes_vec_feret.pickle", "rb") as f:
    runtimes_vec = pickle.load(f)
with open("success_times_feret.pickle", "rb") as f:
    success_times = pickle.load(f)

print("Success Rate: {:.2f}%".format(success_times / len(identity_list) * 100))
print("Average Run Times: {:.2f}".format(np.mean(runtimes_vec)))
print(np.mean(runtimes_vec) * len(identity_list) / success_times)


Success Rate: 7.93%
Average Run Times: 67.72
854.4831460674159
